In [ ]:
class Cola:
    def __init__(self):
        self.citasProcesar = []
        self.tamanio = 0
    def agregarCita(self, cita):
        self.citasProcesar.append(cita)
        print("CITA AGREGADA EXITOSAMENTE")
    def dequeCita(self):
        if self.tamanio == 0:
            return False
        else:
            return self.citasProcesar.pop(0)
    def front(self):
        if self.tamanio == 0:
            return None
        else:
            return self.citasProcesar[0]
    def recorrerCola(self):
        if self.tamanio == 0:
            return "NO HAY CITAS REGISTRADAS EN LA COLA"
        else:
            turno = 1
            for cita in self.citasProcesar:
                print(f'TURNO: {turno}, CITA: {cita}')
                turno += 1

In [ ]:
class PilaHistorial():
    def __init__(self):
        self.operacionesRegistradas = []
        self.tamanioPila = 0
    def agregarOperacion(self, cita, tipoOperacion):
        self.operacionesRegistradas.append((cita,tipoOperacion))
        self.tamanioPila += 1
    def deshacerOperacion(self):
        return self.operacionesRegistradas.pop()
    def peek(self):
        return f'Cita: {self.operacionesRegistradas[self.tamanioPila][0]} Operacion Realizada: {self.operacionesRegistradas[self.tamanioPila][1]}'
    def size(self):
        return self.tamanioPila

In [ ]:
class Nodo:
    def __init__(self, valor):
        self.valor = valor
        self.siguiente = None
    def __str__(self):
        return f'Valor: {self.valor}'

In [ ]:
class ListaEnlazada:
	def __init__(self):
		self.nodoCabeza = None
		self.tamanio = 0
	def push_back(self,valor):
		nuevoNodo = Nodo(valor)
		if self.nodoCabeza == None:
			self.nodoCabeza = nuevoNodo
			self.tamanio += 1
		else:
			nodoActual = self.nodoCabeza
			while nodoActual.siguiente != None:
				nodoActual = nodoActual.siguiente
				nodoActual.siguiente = nuevoNodo
				self.tamanio += 1
	def recorrerLista(self):
		if self.nodoCabeza != None:
			nodoActual = self.nodoCabeza
			while nodoActual != None:
				print(nodoActual,end="--->")
				nodoActual = nodoActual.siguiente
	def deleteNodo(self, pos):
		if self.nodoCabeza == None:
			return False
		if pos > self.tamanio:
			return False
		if pos == 0 and self.tamanio == 1:
			self.nodoCabeza = None
		elif pos == 0:
			siguiente = self.nodoCabeza.siguiente
			self.nodoCabeza = siguiente
			self.tamanio -= 1
		else:
			nodoPrevio = self.nodoCabeza
			nodoSiguiente = self.nodoCabeza.siguiente
			for i in range(pos-1):
				nodoPrevio = nodoSiguiente
				nodoSiguiente = nodoSiguiente.siguiente
				if(nodoSiguiente == None):
					nodoPrevio.siguiente = None
				else:
					nodoPrevio.siguiente = nodoSiguiente.siguiente
					self.tamanio -= 1

In [ ]:
class Cita:
    def  __init__(self, nombrePaciente, hora, nombreDoctor, estado):
        self.nombrePaciente=nombrePaciente
        self.hora=hora
        self.nombreDoctor=nombreDoctor
        self.estado=estado
def registrar_cita():
    paciente = input("Nombre del paciente: ")
    hora = input("Hora de la cita: ")
    doctor = input("Nombre del doctor: ")
    nueva_cita = Cita(paciente, hora, doctor, "Programada")

In [ ]:
print('Menu')
temporal=True
while temporal:
    print('1. Registro de citas\n2. Listado de citas(Pendientes y totales)\n3. Modificacion de citas\n4. Cancelacion de cita\n5. Procesamiento de citas\n6. Deshacer operaciones')
    respuesta=input('Seleccione una opcion')
    if respuesta == 1:
        registrar_cita()
    if 
